In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 745 kB 8.1 MB/s 
  Created wheel for pyngrok: filename=pyngrok-5.1.0-py3-none-any.whl size=19007 sha256=493c6ba1f2514b68ceac216c37106bc1a07334d561d53416428029870cf83a91
  Stored in directory: /root/.cache/pip/wheels/bf/e6/af/ccf6598ecefecd44104069371795cb9b3afbcd16987f6ccfb3
Successfully built pyngrok


In [ ]:
!pip install flask_ngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from flask_ngrok import run_with_ngrok
from flask import Flask,request,render_template
import pickle
import numpy as np
import pandas as pd
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
from pyngrok import ngrok

In [ ]:
!pip install pyngrok

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pyngrok import ngrok

In [ ]:
cd /content/drive/MyDrive/Customer_Transaction_Data

/content/drive/MyDrive/Customer_Transaction_Data


In [ ]:
!ngrok  authtoken 29RJdY1KLnCDjq1FDtAPGyzOgfa_7mMSJF1VhjVX4tmCvBqif

Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml


In [ ]:
import os
from werkzeug.utils import secure_filename

In [ ]:
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename

UPLOAD_FOLDER = '/content/drive/MyDrive/Customer_Transaction_Data/upload_folder'
ALLOWED_EXTENSIONS = {'csv'}

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS

@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        # check if the post request has the file part
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        # if user does not select file, browser also
        # submit an empty part without filename
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return redirect(url_for('uploaded_file',
                                    filename=filename))
    return '''
    <!doctype html>
    <title>Upload new File</title>
    <h1>Upload new File</h1>
    <form method=post enctype=multipart/form-data>
      <input type=file name=file>
      <input type=submit value=Upload>
    </form>
    '''
from flask import send_from_directory

@app.route('/uploads/<filename>')
def uploaded_file(filename):
    return send_from_directory(app.config['UPLOAD_FOLDER'],
                               filename)

In [ ]:
import os
from flask import Flask, flash, request, redirect, url_for
from werkzeug.utils import secure_filename
from flask import send_from_directory
model=pickle.load(open('Customer_transaction_prediction_model.pkl','rb'))
UPLOAD_FOLDER = '/content/drive/MyDrive/Customer_Transaction_Data/upload_folder'
ALLOWED_EXTENSIONS = {'csv'}
app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER
def allowed_file(filename):
    return '.' in filename and \
           filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS
@app.route('/', methods=['GET', 'POST'])
def upload_file():
    if request.method == 'POST':
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No selected file')
            return redirect(request.url)
        if file and allowed_file(file.filename):
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
            return send_from_directory(app.config['UPLOAD_FOLDER'],
                               filename)
    return render_template('bank_prediction.html')

In [ ]:
@app.route('/getprediction',methods=['GET', 'POST'])
def getprediction():
  uploaded_file_name =os.listdir('/content/drive/MyDrive/Customer_Transaction_Data/upload_folder')
  filepath=os.path.join('/content/drive/MyDrive/Customer_Transaction_Data/upload_folder/',uploaded_file_name[2])
  test_data=pd.read_csv(filepath)
  real_test=pickle.load(open('/content/drive/MyDrive/Customer_Transaction_Data/real_test_datset.pkl','rb'))
  fake_test=pickle.load(open('/content/drive/MyDrive/Customer_Transaction_Data/fake_test_dataset.pkl','rb'))
  Customer_id=test_data['ID_code']
  mathematical_features=test_data.iloc[:,2:]
  mathematical_features['sum']=mathematical_features.sum(axis=1)
  mathematical_features['min']=mathematical_features.min(axis=1)
  mathematical_features['max']=mathematical_features.max(axis=1)
  mathematical_features['mean']=mathematical_features.mean(axis=1)
  mathematical_features['std']=mathematical_features.std(axis=1)
  mathematical_features['skew']=mathematical_features.skew(axis=1)
  mathematical_features['kurt']=mathematical_features.kurtosis(axis=1)
  mathematical_features['med']=mathematical_features.median(axis=1)
  mathematical_features.insert(loc=0, column='ID_code', value=Customer_id)
  check_real_features=real_test.loc[real_test['ID_code'].isin(list(Customer_id))]
  column_names=[f"var_{i}_unique" for i in range(200)]
  unique_real_features=check_real_features[column_names]#selecting unique columns
  check_fake_features=fake_test.loc[fake_test['ID_code'].isin(list(Customer_id))]
  unique_fake_features=check_fake_features[column_names]#selecting unique columns
  f=[unique_real_features,unique_fake_features]
  unique_featurized_testdata=pd.concat(f)
  z=[test_data,mathematical_features,unique_featurized_testdata]
  final_test_data=pd.concat(z,axis=1, join="inner")
  final_test=final_test_data.drop(['ID_code'],axis=1)
  prediction=model.predict(final_test)
  msg=[]
  for i in range(len(prediction)):
    if prediction[i]>=0.5:
      msg.append('customer will make transaction in future')
    else :
      msg.append('customer will  not make transaction in future')
  return render_template('bank_prediction.html',output=msg)


In [ ]:
run_with_ngrok(app)
#!ngrok http 80
app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://5609-34-86-101-145.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040


127.0.0.1 - - [02/Jun/2022 19:50:15] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [02/Jun/2022 19:50:17] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [02/Jun/2022 19:50:26] "POST /getprediction HTTP/1.1" 200 -
